In [1]:
import base64
import base58
import zlib
import json
import html
import codecs
import brotli
from urllib.parse import quote_plus
from lzstring import LZString
from urllib.parse import urlparse
import pandas as pd
import urllib
import hashlib
import mmh3
import mmh as mmhash
import base64
import base58
import zlib
import json
import re
from urllib.parse import quote_plus, unquote_plus
import codecs

from lzstring import LZString
import html
import brotli
import lzma
import brotli



def rot_n(text, n):
    result = []
    for char in text:
        if 'a' <= char <= 'z':
            result.append(chr((ord(char) - ord('a') + n) % 26 + ord('a')))
        elif 'A' <= char <= 'Z':
            result.append(chr((ord(char) - ord('A') + n) % 26 + ord('A')))
        else:
            result.append(char)
    return ''.join(result)


def compress_with_zlib(compression_type, string, level=6):
    if compression_type == 'deflate':
        compressor = zlib.compressobj(level, zlib.DEFLATED, -zlib.MAX_WBITS)
    elif compression_type == 'zlib':
        compressor = zlib.compressobj(level, zlib.DEFLATED, zlib.MAX_WBITS)
    elif compression_type == 'gzip':
        compressor = zlib.compressobj(level, zlib.DEFLATED, zlib.MAX_WBITS | 16)
    else:
        raise ValueError(f"Unsupported zlib compression format: {compression_type}")
    return compressor.compress(string) + compressor.flush()


def create_variations(input_string):
    x_bytes = input_string.encode('utf-8')
    x_str = input_string
    results = {}

    encodings = {
        'base16': lambda x: base64.b16encode(x),
        'base32': lambda x: base64.b32encode(x),
        'base58': lambda x: base58.b58encode(x),
        'base64': lambda x: base64.b64encode(x),
        'urlencode': lambda x: quote_plus(x_str),
        'deflate': lambda x: compress_with_zlib('deflate', x),
        'zlib': lambda x: compress_with_zlib('zlib', x),
        'gzip': lambda x: compress_with_zlib('gzip', x), 
        'binary': lambda x: ''.join(format(b, '08b') for b in x),
        'entity': lambda x: html.escape(x_str),
        'rot13': lambda x: codecs.encode(x_str, 'rot_13'),
        'deflate_base64': lambda x: base64.b64encode(compress_with_zlib('deflate', x)),
        'deflateraw_base64': lambda x: base64.b64encode(compress_with_zlib('deflate', x)),
        'gzip_base64': lambda x: base64.b64encode(compress_with_zlib('gzip', x)),
        'brotli_base64': lambda x: base64.b64encode(brotli.compress(x)),
        'deflate_hex': lambda x: compress_with_zlib('deflate', x).hex(),
        'gzip_hex': lambda x: compress_with_zlib('gzip', x).hex(),
        'zlib64': lambda x: base64.b64encode(zlib.compress(x)).decode('utf-8')
    }  

    # Optional encodings (wrapped in try-except)
    try:
        lz = LZString()
        encodings['lz64'] = lambda x: base64.b64encode(lz.compress(x_str).encode('utf-8'))
        encodings['lz64de'] = lambda x: base64.b64encode(lz.compress(x_str).encode('utf-8')).decode()
        encodings['lz_string'] = lambda x: lz.compressToEncodedURIComponent(x_str)
    except Exception:
        pass

    try:
        import lzw
        encodings['lzw'] = lambda x: json.dumps(lzw.encode(x_str))
    except Exception:
        pass
    
    hashes = dict()
    # hashes['md2'] = lambda x: self._get_md2_hash(x)
    encodings['md5'] = lambda x: hashlib.md5(x).hexdigest()
    encodings['sha1'] = lambda x: hashlib.sha1(x).hexdigest()
    encodings['sha256'] = lambda x: hashlib.sha256(x).hexdigest()
    encodings['sha224'] = lambda x: hashlib.sha224(x).hexdigest()
    encodings['sha384'] = lambda x: hashlib.sha384(x).hexdigest()
    encodings['sha512'] = lambda x: hashlib.sha512(x).hexdigest()
    encodings['sha3_224'] = lambda x: hashlib.sha3_224(x if isinstance(x, bytes) else x.encode('utf-8')).hexdigest()
    encodings['sha3_256'] = lambda x: hashlib.sha3_256(x if isinstance(x, bytes) else x.encode('utf-8')).hexdigest()
    encodings['sha3_384'] = lambda x: hashlib.sha3_384(x if isinstance(x, bytes) else x.encode('utf-8')).hexdigest()
    encodings['sha3_512'] = lambda x: hashlib.sha3_512(x if isinstance(x, bytes) else x.encode('utf-8')).hexdigest()
    encodings['mmh3_32'] = lambda x: str(mmh3.hash(x))
    encodings['mmh3_64_1'] = lambda x: str(mmh3.hash64(x)[0])
    encodings['mmh3_64_2'] = lambda x: str(mmh3.hash64(x)[1])
    encodings['mmh3_128'] = lambda x: str(mmh3.hash128(x))
    # hashes['blake2b'] = lambda x: pyblake2.blake2b(x).hexdigest()
    # hashes['blake2s'] = lambda x: pyblake2.blake2s(x).hexdigest()
    encodings['crc32'] = lambda x: str(zlib.crc32(x) & 0xFFFFFFFF)
    encodings['adler32'] = lambda x: str(zlib.adler32(x))
 
    
    
    # Apply all encodings
    for name, func in encodings.items():
        try:
            result = func(x_bytes)
            if isinstance(result, bytes):
                try:
                    results[name] = result.decode('utf-8')
                except UnicodeDecodeError:
                    results[name] = result.hex()
            else:
                results[name] = result
        except Exception as e:
            results[name] = f'ERROR: {str(e)}'
            
    return results


# 🔽 Run and print results
encoded = create_variations("hello world")
for k, v in encoded.items():
    print(f"{k}: {v}")

base16: 68656C6C6F20776F726C64
base32: NBSWY3DPEB3W64TMMQ======
base58: StV1DL6CwTryKyV
base64: aGVsbG8gd29ybGQ=
urlencode: hello+world
deflate: cb48cdc9c95728cf2fca490100
zlib: 789ccb48cdc9c95728cf2fca4901001a0b045d
gzip: 1f8b0800000000000013cb48cdc9c95728cf2fca49010085114a0d0b000000
binary: 0110100001100101011011000110110001101111001000000111011101101111011100100110110001100100
entity: hello world
rot13: uryyb jbeyq
deflate_base64: y0jNyclXKM8vykkBAA==
deflateraw_base64: y0jNyclXKM8vykkBAA==
gzip_base64: H4sIAAAAAAAAE8tIzcnJVyjPL8pJAQCFEUoNCwAAAA==
brotli_base64: CwWAaGVsbG8gd29ybGQD
deflate_hex: cb48cdc9c95728cf2fca490100
gzip_hex: 1f8b0800000000000013cb48cdc9c95728cf2fca49010085114a0d0b000000
zlib64: eJzLSM3JyVcozy/KSQEAGgsEXQ==
lz64: 1oXjgLbmg7ZA7rqQ4pyw04gA
lz64de: 1oXjgLbmg7ZA7rqQ4pyw04gA
lz_string: BYUwNmD2AEDukCcwBMg
md5: 5eb63bbbe01eeed093cb22bb8f5acdc3
sha1: 2aae6c35c94fcfb415dbe95f408b9ce91ee846ed
sha256: b94d27b9934d3e08a52e52d7da7dabfac484efe37a5380ee9088f7ace2efcde9
sha2

In [2]:
from urllib.parse import urlparse
import pandas as pd
import urllib
import hashlib
import mmh3
import mmh as mmhash
import base64
import base58
import zlib
import json
import re
from urllib.parse import quote_plus, unquote_plus
import codecs

from lzstring import LZString
import html
import brotli
import lzma
import brotli


# DELIMITERS = re.compile('[&|\,]')
DELIMITERS = re.compile(r'[&|,=]')

EXTENSION_RE = re.compile('\.[A-Za-z]{2,4}$')
ENCODING_LAYERS = 3
ENCODINGS_NO_ROT = ['base64', 'base32', 'base16',
    'base58', 'zlib', 'gzip', 'urlencode',
    'entity', 'zlib64', 'brotli_base64', 'json',
    'deflate_base64', 'deflateraw_base64', 'gzip_base64',
    'hex' ]
LIKELY_ENCODINGS = ['base16', 'base32', 'base58', 'base64',
                    'urlencode',  'entity', 'zlib64']
HASHES = [  'md5', 'sha1', 'sha256', 'sha224', 'sha384',
          'sha512', 'sha3_224', 'sha3_256', 'sha3_384', 'sha3_512',   'mmh3_32', 'mmh3_64_1', 'mmh3_64_2', 'mmh3_128',  ]


class Hasher():
    def __init__(self):
        # Define Supported hashes
        hashes = dict()
        # hashes['md2'] = lambda x: self._get_md2_hash(x)
        hashes['md5'] = lambda x: hashlib.md5(x).hexdigest()
        hashes['sha1'] = lambda x: hashlib.sha1(x).hexdigest()
        hashes['sha256'] = lambda x: hashlib.sha256(x).hexdigest()
        hashes['sha224'] = lambda x: hashlib.sha224(x).hexdigest()
        hashes['sha384'] = lambda x: hashlib.sha384(x).hexdigest()
        hashes['sha512'] = lambda x: hashlib.sha512(x).hexdigest()
        hashes['sha3_224'] = lambda x: hashlib.sha3_224(x if isinstance(x, bytes) else x.encode('utf-8')).hexdigest()
        hashes['sha3_256'] = lambda x: hashlib.sha3_256(x if isinstance(x, bytes) else x.encode('utf-8')).hexdigest()
        hashes['sha3_384'] = lambda x: hashlib.sha3_384(x if isinstance(x, bytes) else x.encode('utf-8')).hexdigest()
        hashes['sha3_512'] = lambda x: hashlib.sha3_512(x if isinstance(x, bytes) else x.encode('utf-8')).hexdigest()
        hashes['mmh3_32'] = lambda x: str(mmh3.hash(x))
        hashes['mmh3_64_1'] = lambda x: str(mmh3.hash64(x)[0])
        hashes['mmh3_64_2'] = lambda x: str(mmh3.hash64(x)[1])
        hashes['mmh3_128'] = lambda x: str(mmh3.hash128(x))
       # hashes['blake2b'] = lambda x: pyblake2.blake2b(x).hexdigest()
       # hashes['blake2s'] = lambda x: pyblake2.blake2s(x).hexdigest()
        hashes['crc32'] = lambda x: str(zlib.crc32(x) & 0xFFFFFFFF)
        hashes['adler32'] = lambda x: str(zlib.adler32(x))

        self._hashes = hashes
        self.hashes_and_checksums = list(self._hashes.keys())
        self.supported_hashes = HASHES

    def _get_hashlib_hash(self, name, string):
        """Use for hashlib hashes that don't have a shortcut"""
        hasher = hashlib.new(name)
        hasher.update(string)
        return hasher.hexdigest()

    def _get_md2_hash(self, string):
        """Compute md2 hash"""
        md2 = MD2.new()
        md2.update(string)
        return md2.hexdigest()

    def get_hash(self, hash_name, string):
        """Compute the desired hash"""
        return self._hashes[hash_name](string)


class Encoder():
    def __init__(self):
        # Define supported encodings
        encodings = dict()
        encodings['base16'] = lambda x: base64.b16encode(x)
        encodings['base32'] = lambda x: base64.b32encode(x)
        encodings['base58'] = lambda x: base58.b58encode(x)
        encodings['base64'] = lambda x: base64.b64encode(x)
        encodings['urlencode'] = lambda x: quote_plus(x)
        encodings['deflate'] = lambda x: self._compress_with_zlib('deflate', x)
        encodings['zlib'] = lambda x: self._compress_with_zlib('zlib', x)
        encodings['gzip'] = lambda x: self._compress_with_zlib('gzip', x)
        encodings['json'] = lambda x: json.dumps(x)
        encodings['binary'] = lambda x: ''.join(format(ord(c), '08b') for c in x)
        encodings['entity'] = lambda x: html.escape(x)
        encodings['rot13'] = lambda x: codecs.encode(x, 'rot_13')
        
        for i in range(1, 26):
            if i == 13:
                continue  # handled separately with codecs
            encodings[f'rot{i}'] = lambda x, i=i: rot_n(x, i)
 
        encodings['deflate_base64'] = lambda x: base64.b64encode(zlib.compress(x))
        encodings['deflateraw_base64'] = lambda x: base64.b64encode(zlib.compress(x, -zlib.MAX_WBITS))
        encodings['gzip_base64'] = lambda x: base64.b64encode(zlib.compress(x, zlib.MAX_WBITS | 16))
        encodings['brotli_base64'] = lambda x: base64.b64encode(brotli.compress(x))
        encodings['deflate_hex'] = lambda x: zlib.compress(x).hex()
        encodings['gzip_hex'] = lambda x: zlib.compress(x, zlib.MAX_WBITS | 16).hex()
        
        
        try:
            lz = LZString()
            encodings['lz64'] = lambda x: base64.b64encode(lz.compress(x.decode('utf-8')).encode('utf-8'))
        except:
            pass
        
        try:
            lz = LZString()
            encodings['lz64de'] = lambda x: base64.b64encode(LZString().compress(x.decode('utf-8')).encode('utf-8')).decode('utf-8')
        except:
            pass
        
        encodings['zlib64'] = lambda x: base64.b64encode(zlib.compress(x.encode('utf-8') if isinstance(x, str) else x)).decode('utf-8')
        
        try:
            import lzw
            encodings['lzw'] = lambda x: json.dumps(lzw.encode(x.decode('utf-8')))
        except:
            pass
        try:
            lz = LZString()
            encodings['lz_string'] = lambda x: lz.compressToEncodedURIComponent(x.decode('utf-8'))
        except:
            pass
        self._encodings = encodings
        self.supported_encodings = list(self._encodings.keys())

    def _compress_with_zlib(self, compression_type, string, level=6):
        """Compress in one of the zlib supported formats: zlib, gzip, or deflate.
        For a description see: http://stackoverflow.com/a/22311297/6073564
        """
        if compression_type == 'deflate':
            compressor = zlib.compressobj(level, zlib.DEFLATED,
                                          -zlib.MAX_WBITS)
        elif compression_type == 'zlib':
            compressor = zlib.compressobj(level, zlib.DEFLATED,
                                          zlib.MAX_WBITS)
        elif compression_type == 'gzip':
            compressor = zlib.compressobj(level, zlib.DEFLATED,
                                          zlib.MAX_WBITS | 16)
        else:
            raise ValueError("Unsupported zlib compression format %s." %
                             compression_type)
        return compressor.compress(string) + compressor.flush()

    def encode(self, encoding, string):
        """Encode `string` in desired `encoding`"""
        return self._encodings[encoding](string)


class DecodeException(Exception):
    def __init__(self, message, error):
        super(DecodeException, self).__init__(message)
        self.error = error


def rot_n(text, n):
    def rotate_char(c):
        if 'a' <= c <= 'z':
            return chr((ord(c) - ord('a') + n) % 26 + ord('a'))
        elif 'A' <= c <= 'Z':
            return chr((ord(c) - ord('A') + n) % 26 + ord('A'))
        return c
    return ''.join(rotate_char(c) for c in text)

class Decoder():
    def __init__(self):
        # Define supported encodings
        decodings = dict()
        decodings['base16'] = lambda x: base64.b16decode(x)
        decodings['base32'] = lambda x: base64.b32decode(x)
        decodings['base58'] = lambda x: base58.b58decode(x)
        decodings['base64'] = lambda x: base64.b64decode(x)
        decodings['urlencode'] = lambda x: unquote_plus(x)
        decodings['deflate'] = lambda x: self._decompress_with_zlib('deflate',
                                                                    x)
        decodings['zlib'] = lambda x: self._decompress_with_zlib('zlib', x)
        decodings['gzip'] = lambda x: self._decompress_with_zlib('gzip', x)
        decodings['json'] = lambda x: json.loads(x)
        decodings['binary'] = lambda x: ''.join(chr(int(x[i:i+8], 2)) for i in range(0, len(x), 8))
        decodings['entity'] = lambda x: html.unescape(x)
        
        decodings['rot13'] = lambda x: codecs.decode(x, 'rot_13')
        
        for i in range(1, 26):
            if i == 13:
                continue  # handled separately with codecs
            decodings[f'rot{i}'] = lambda x, i=i: rot_n(x, i)
            
        try:
            lz = LZString()
            decodings['lz64'] = lambda x: lz.decompress(base64.b64decode(x).decode('utf-8'))
        except:
            pass
        
        decodings['zlib64'] = lambda x: zlib.decompress(base64.b64decode(x)).decode('utf-8', errors='ignore')
        
        try:
            lz = LZString()
            decodings['lz64de'] = lambda x: LZString().decompress(base64.b64decode(x).decode('utf-8'))
        except:
            pass

        try:
            lz = LZString()
            decodings['lz_string'] = lambda x: lz.decompressFromEncodedURIComponent(x)
        except:
            pass

        try:
            
            import lzw
            decodings['lzw'] = lambda x: ''.join(lzw.decode(json.loads(x)))
        except:
            pass

        try:
            decodings['brotli_base64'] = lambda x: brotli.decompress(base64.b64decode(x)).decode('utf-8', errors='ignore')
        except:
            pass

        decodings['deflate_base64'] = lambda x: zlib.decompress(base64.b64decode(x), -zlib.MAX_WBITS).decode('utf-8', errors='ignore')
        decodings['deflateraw_base64'] = lambda x: zlib.decompress(base64.b64decode(x), -zlib.MAX_WBITS).decode('utf-8', errors='ignore')
        decodings['gzip_base64'] = lambda x: zlib.decompress(base64.b64decode(x), zlib.MAX_WBITS | 16).decode('utf-8', errors='ignore')
        decodings['deflate_hex'] = lambda x: zlib.decompress(bytes.fromhex(x)).decode('utf-8', errors='ignore')
        decodings['gzip_hex'] = lambda x: zlib.decompress(bytes.fromhex(x), zlib.MAX_WBITS | 16).decode('utf-8', errors='ignore')
 
        self._decodings = decodings
        self.supported_encodings = list(self._decodings.keys())

    def _decompress_with_zlib(self, compression_type, string, level=9):
        """Compress in one of the zlib supported formats: zlib, gzip, or deflate.
        For a description see: http://stackoverflow.com/a/22311297/6073564
        """
        if compression_type == 'deflate':
            return zlib.decompress(string, -zlib.MAX_WBITS)
        elif compression_type == 'zlib':
            return zlib.decompress(string, zlib.MAX_WBITS)
        elif compression_type == 'gzip':
            return zlib.decompress(string, zlib.MAX_WBITS | 16)
        else:
            raise ValueError("Unsupported zlib compression format %s." %
                             compression_type)

    def decode_error(self):
        """Catch-all error for all supported decoders"""

    def decode(self, encoding, string):
        """Decode `string` encoded by `encoding`"""
        try:
            return self._decodings[encoding](string)
        except Exception as e:
            raise DecodeException(
                'Error while trying to decode %s' % encoding,
                e
            )


class LeakDetector():
    def __init__(self, search_strings, precompute_hashes=True, hash_set=None,
                 hash_layers=2, precompute_encodings=True, encoding_set=None,
                 encoding_layers=2, debugging=False):
        """LeakDetector searches URL, POST bodies, and cookies for leaks.

        The detector is constructed with a set of search strings (given by
        the `search_strings` parameters. It has several methods to check for
        leaks containing these strings in URLs, POST bodies, and cookie header
        strings.

        Parameters
        ==========
        search_strings : list
            LeakDetector will search for leaks containing any item in this list
        precompute_hashes : bool
            Set to `True` to include precomputed hashes in the candidate set.
        hash_set : list
            List of hash functions to use when building the set of candidate
            strings.
        hash_layers : int
            The detector will find instances of `search_string` iteratively
            hashed up to `hash_layers` times by any combination of supported
            hashes.
        precompute_encodings : bool
            Set to `True` to include precomputed encodings in the candidate set
        encoding_set : list
            List of encodings to use when building the set of candidate
            strings.
        encoding_layers : int
            The detector will find instances of `search_string` iteratively
            encoded up to `encoding_layers` times by any combination of
            supported encodings.
        debugging : bool
            Set to `True` to enable a verbose output.
        """
        self.search_strings = search_strings
        self._min_length = min([len(x) for x in search_strings])
        self._hasher = Hasher()
        self._hash_set = hash_set
        self._hash_layers = hash_layers
        self._encoder = Encoder()
        self._encoding_set = encoding_set
        self._encoding_layers = encoding_layers
        self._decoder = Decoder()
        self._precompute_pool = dict()
        # If hash/encoding sets aren't specified, use all available.
        if self._hash_set is None:
            self._hash_set = self._hasher.supported_hashes
        if self._encoding_set is None:
            self._encoding_set = self._encoder.supported_encodings
        self._build_precompute_pool(precompute_hashes, precompute_encodings)
        self._debugging = debugging

    def _compute_hashes(self, string, layers, prev_hashes=tuple()):
        """Returns all iterative hashes of `string` up to the
        specified number of `layers`"""
        for h in self._hasher.supported_hashes:
            hashed_string = self._hasher.get_hash(h, string.encode('utf-8'))
            if hashed_string == string:  # skip no-ops
                continue
            hash_stack = (h,) + prev_hashes
            self._precompute_pool[hashed_string] = hash_stack
            if layers > 1:
                self._compute_hashes(hashed_string, layers-1, hash_stack)

    def _compute_encodings(self, string, layers, prev_encodings=tuple()):
        for enc in self._encoding_set:
            try:
                input_data = string.encode('utf-8') if enc in ['base16', 'base32', 'base58', 'base64', 'deflate', 'zlib', 'gzip'] else string
                encoded = self._encoder.encode(enc, input_data)
                if isinstance(encoded, bytes):
                    encoded_string = encoded.decode('utf-8', errors='ignore')
                else:
                    encoded_string = str(encoded)
            except Exception:
                continue
            if encoded_string == string:
                continue
            encoding_stack = (enc,) + prev_encodings
            self._precompute_pool[encoded_string] = encoding_stack
            if layers > 1:
                self._compute_encodings(encoded_string, layers - 1, encoding_stack)

    def _build_precompute_pool(self, precompute_hashes, precompute_encodings):
        """Build a pool of hashes for the given search string"""
        seed_strings = list()
        for string in self.search_strings:
            seed_strings.append(string)
            if string.startswith('http'):
                continue
            all_lower = string.lower()
            if all_lower != string:
                seed_strings.append(string.lower())
            all_upper = string.upper()
            if all_upper != string:
                seed_strings.append(string.upper())

        strings = list()
        for string in seed_strings:
            strings.append(string)
            # If the search string appears to be an email address, we also want
            # to include just the username portion of the URL, and the address
            # and username with any '.'s removed from the username (since these
            # are optional in Gmail).
            if '@' in string:
                parts = string.rsplit('@')
                if len(parts) == 2:
                    uname, domain = parts
                    strings.append(uname)
                    strings.append(re.sub('\.', '', uname))
                    strings.append(re.sub('\.', '', uname) + '@' + domain)
                # Domain searches have too many false positives
                # strings.append(parts[1])
                # strings.append(parts[1].rsplit('.', 1)[0])
            # The URL tokenizer strips file extensions. So if our search string
            # has a file extension we should also search for a stripped version
            if re.match(EXTENSION_RE, string):
                strings.append(re.sub(EXTENSION_RE, '', string))
        for string in strings:
            self._precompute_pool[string] = (string,)
        self._min_length = min([len(x) for x in self._precompute_pool.keys()])
        initial_items = list(self._precompute_pool.items())
        if precompute_hashes:
            for string, name in initial_items:
                self._compute_hashes(string, self._hash_layers, name)
        if precompute_encodings:
            for string, name in initial_items:
                self._compute_encodings(string, self._encoding_layers, name)

    def _split_on_delims(self, string, rv_parts, rv_named):
        """Splits a string on several delimiters"""
        if string == '':
            return
        parts = set(re.split(DELIMITERS, string))
        if '' in parts:
            parts.remove('')
        for part in parts:
            if part == '':
                continue
            count = part.count('=')
            if count != 1:
                rv_parts.add(part)
            if count == 0:
                continue
            n, k = part.split('=', 1)
            if len(n) > 0 and len(k) > 0:
                rv_named.add((n, k))
            else:
                rv_parts.add(part)

    def check_if_in_precompute_pool(self, string):
        """Returns a tuple that lists the (possibly layered) hashes or
        encodings that result in input string
        """
        try:
            return self._precompute_pool[str(string)]
        except KeyError:
            return
        except (UnicodeDecodeError, UnicodeEncodeError):
            return

    def check_for_leak(self, string, layers=1, prev_encodings=tuple(),
                       prev=''):
        """Check if given string contains a leak"""
        # Short tokens won't contain email address
        if len(string) < self._min_length:
            return

        # Check if direct hash or plaintext
        rv = self.check_if_in_precompute_pool(string)
        if rv is not None:
            return prev_encodings + rv

        # Try encodings
        for encoding in self._encoding_set:
            # multiple rots are unnecessary
            if encoding.startswith('rot') and prev.startswith('rot'):
                continue
            try:
                decoded = self._decoder.decode(encoding, string)
                if type(decoded) == int:
                    decoded = str(decoded)
            except DecodeException:  # means this isn't the correct decoding
                continue
            if decoded == string:  # don't add no-ops
                continue
            if decoded is None:  # Empty decodings aren't useful
                continue
            encoding_stack = prev_encodings + (encoding,)
            if layers > 1:
                rv = self.check_for_leak(decoded, layers-1,
                                         encoding_stack, encoding)
                if rv is not None:
                    return rv
            else:
                # New: allow substring matches
                for known_string, transform_stack in self._precompute_pool.items():
                    if known_string in str(decoded):
                        return encoding_stack + transform_stack
        return

    def _check_parts_for_leaks(self, tokens, parameters, nlayers):
        """Check token and parameter string parts for leaks"""
        leaks = list()
        for token in tokens:
            leak = self.check_for_leak(token, layers=nlayers)
            if leak is not None:
                leaks.append(leak)
        for name, value in parameters:
            leak = self.check_for_leak(value, layers=nlayers)
            if leak is not None:
                leaks.append(leak)
            leak = self.check_for_leak(name, layers=nlayers)
            if leak is not None:
                leaks.append(leak)
        return leaks

    def _split_url(self, url):
        """Split url path and query string on delimiters"""
        tokens = set()
        parameters = set()
        try:
            purl = urlparse(url)
        except ValueError:
            print(f"Can't parse url: {url}")
            return [], []
        path_parts = purl.path.split('/')
        for part in path_parts:
            if not part.endswith('.com'):
                part = re.sub(EXTENSION_RE, '', part)
            self._split_on_delims(part, tokens, parameters)
        self._split_on_delims(purl.query, tokens, parameters)
        self._split_on_delims(purl.fragment, tokens, parameters)
        return tokens, parameters

    def check_url(self, url, encoding_layers=3, substring_search=True):
        """Check if a given url contains a leak"""
        tokens, parameters = self._split_url(url)
        if self._debugging:
            print("URL tokens:") 
            print(tokens)
            for token in tokens:
                print(token)
            print("\nURL parameters:")
            for key, value in parameters:
                print("Key: %s | Value: %s" % (key, value))
        return self._check_whole_and_parts_for_leaks(
            url, tokens, parameters, encoding_layers, substring_search)

    def _get_header_str(self, header_str, header_name):
        """Returns the header string parsed from `header_str`"""
        for item in json.loads(header_str):
            if item[0] == header_name:
                return item[1]
        return ""

    def _split_cookie(self, cookie_str, from_request=True):
        """Returns all parsed parts of the cookie names and values"""
        tokens = set()
        parameters = set()
        try:
            if from_request:
                cookies = ck.Cookies.from_request(cookie_str)
            else:
                cookies = ck.Cookies.from_response(cookie_str,
                                                   ignore_bad_cookies=True)
        except (ck.InvalidCookieError, UnicodeDecodeError, KeyError):
            return tokens, parameters  # return empty sets

        for cookie in cookies.values():
            self._split_on_delims(cookie.name, tokens, parameters)
            self._split_on_delims(cookie.value, tokens, parameters)
        return tokens, parameters

    def get_location_str(self, header_str):
        return self._get_header_str(header_str, "Location")

    def get_referrer_str(self, header_str):
        return self._get_header_str(header_str, "Referer")

    def get_cookie_str(self, header_str, from_request=True):
        if not header_str:
            return ""
        if from_request:
            header_name = 'Cookie'
        else:
            header_name = 'Set-Cookie'

        return self._get_header_str(header_str, header_name)

    def check_cookies(self, header_str, encoding_layers=3,
                      from_request=True, substring_search=True):
        """Check the cookies portion of the header string for leaks"""
        cookie_str = self.get_cookie_str(header_str, from_request)
        if not cookie_str:
            return list()
        tokens, parameters = self._split_cookie(header_str, from_request=from_request)
        return self._check_whole_and_parts_for_leaks(
            cookie_str, tokens, parameters, encoding_layers, substring_search)

    def check_location_header(self, location_str, encoding_layers=3,
                              substring_search=True):
        """Check the Location HTTP response header for leaks."""
        if location_str == '':
            return list()
        tokens, parameters = self._split_url(location_str)
        return self._check_whole_and_parts_for_leaks(
            location_str, tokens, parameters, encoding_layers,
            substring_search)

    def check_referrer_header(self, header_str, encoding_layers=3,
                              substring_search=True):
        """Check the Referer HTTP request header for leaks."""
        if header_str == '':
            return list()
        referrer_str = self.get_referrer_str(header_str)
        # We use this check instead of ==''
        # since _get_header_str may return None
        if not referrer_str:
            return list()
        tokens, parameters = self._split_url(referrer_str)
        return self._check_whole_and_parts_for_leaks(
            referrer_str, tokens, parameters, encoding_layers,
            substring_search)
    

    def _extract_url_style_payload(self, payload_str, tokens, parameters, seen, depth):
            """
            Extract tokens and named parameters from a URL-style payload.
            Example: en=Scroll_depth&epn.scroll_depth=30&ep.url=https%3A%2F%2Fexample.com
            """
            if depth > 3 or payload_str in seen:
                return
            seen.add(payload_str)

            try:
                pairs = payload_str.split('&')
                for pair in pairs:
                    if '=' in pair:
                        k, v = pair.split('=', 1)
                        k = k.strip()
                        v = unquote_plus(v.strip())
                        tokens.add(k)
                        tokens.add(v)
                        parameters.add((k, v))

                        # Recursively extract deeper from values
                        self._split_on_delims(k, tokens, parameters)
                        self._split_on_delims(v, tokens, parameters)

                        # Recurse into value for deeper encoding
                        for enc in LIKELY_ENCODINGS:
                            try:
                                decoded = self._decoder.decode(enc, v)
                                if isinstance(decoded, bytes):
                                    decoded = decoded.decode("utf-8", errors="ignore")
                                decoded = decoded.strip()
                                if decoded and decoded != v:
                                    self._extract_url_style_payload(decoded, tokens, parameters, seen, depth + 1)
                            except Exception:
                                continue
                    else:
                        tokens.add(pair)
            except Exception as e:
                if self._debugging:
                    print(f"Error parsing URL-style payload: {e}")
           
    def _extract_json_array_payload(self, payload_str, tokens, parameters, seen, depth):
        """
        Extract tokens and values from a JSON array-style payload.
        Handles deeply nested arrays.
        """
        if depth > 3 or payload_str in seen:
            return
        seen.add(payload_str)

        try:
            data = json.loads(payload_str)
            if not isinstance(data, list):
                return

            def recurse_array(arr, depth):
                if depth > 3:
                    return
                for item in arr:
                    if isinstance(item, list):
                        recurse_array(item, depth + 1)
                    elif isinstance(item, dict):
                        for k, v in item.items():
                            k = str(k).strip()
                            v = str(v).strip()
                            tokens.add(k)
                            tokens.add(v)
                            parameters.add((k, v))
                            self._split_on_delims(k, tokens, parameters)
                            self._split_on_delims(v, tokens, parameters)
                    elif item is not None:
                        val = str(item).strip()
                        tokens.add(val)
                        self._split_on_delims(val, tokens, parameters)

            recurse_array(data, 0)

        except Exception as e:
            if self._debugging:
                print(f"Error parsing JSON array payload: {e}")
    def _extract_json_object_payload(self, payload_str, tokens, parameters, seen, depth):
        """
        Extract tokens and key-value pairs from a JSON object-style payload.
        Handles deeply nested dictionaries.
        """
        if depth > 3 or payload_str in seen:
            return
        seen.add(payload_str)

        try:
            data = json.loads(payload_str)
            if not isinstance(data, dict):
                return

            def recurse_obj(obj, depth):
                if depth > 3:
                    return
                for k, v in obj.items():
                    k_str = str(k).strip()
                    tokens.add(k_str)

                    if isinstance(v, dict):
                        recurse_obj(v, depth + 1)
                    elif isinstance(v, list):
                        for item in v:
                            if isinstance(item, (dict, list)):
                                recurse_obj(item, depth + 1)
                            else:
                                val = str(item).strip()
                                tokens.add(val)
                                parameters.add((k_str, val))
                                self._split_on_delims(val, tokens, parameters)
                    elif v is not None:
                        val = str(v).strip()
                        tokens.add(val)
                        parameters.add((k_str, val))
                        self._split_on_delims(k_str, tokens, parameters)
                        self._split_on_delims(val, tokens, parameters)

            recurse_obj(data, 0)

        except Exception as e:
            if self._debugging:
                print(f"Error parsing JSON object payload: {e}")
                
    def check_payload(self, payload_str, encoding_layers=3, substring_search=True):
        tokens = set()
        parameters = set()
        seen_payloads = set()
        
        

      #  self._extract_url_style_payload(payload_str, tokens, parameters, seen_payloads, depth=0)
       # self._extract_json_array_payload(payload_str, tokens, parameters, seen_payloads, depth=0)
       # self._extract_json_object_payload(payload_str, tokens, parameters, seen_payloads, depth=0)

        if self._debugging:
            print("Tokens:", tokens)
            print("Parameters:", parameters)

        return self._check_whole_and_parts_for_leaks(
            payload_str, tokens, parameters, encoding_layers, substring_search
        )
           
    def _check_whole_and_parts_for_leaks(self, input_string, tokens,
                                         parameters, encoding_layers,
                                         substring_search):
        """Search an input string and its parts for leaks."""
        results = self._check_parts_for_leaks(tokens, parameters,
                                              encoding_layers)
        if substring_search:
            substr_results = self.substring_search(input_string, max_layers=2)
            # filter repeating results
            return list(set(results + substr_results))
        else:
            return results

    def substring_search(self, input_string, max_layers=None):
        """Do a substring search for all precomputed hashes/encodings

        `max_layers` limits the number of encoding/hashing layers used in the
        substring search (to limit time). The default is no limit (`None`).
        """
        if input_string is None or input_string == '':
            return list()
        try:
            input_string = input_string.encode('utf8')
        except (UnicodeDecodeError, UnicodeEncodeError):
            print(f"ERROR encoding: {repr(input_string)}")
            return list()
        leaks = list()
        for string, transform_stack in self._precompute_pool.items():
            if max_layers and len(transform_stack) > (max_layers + 1):
                continue
            if string.encode('utf-8', errors='ignore') in input_string:
                leaks.append(transform_stack)
        return leaks
    

 


if __name__ == "__main__":
    # Sample value to detect
    secret = "hi_my"

    # Create the leak detector with one known secret string
    detector = LeakDetector(search_strings=[secret], debugging=True)

    # Example URL containing a base64-encoded version of the secret
    import base64
    encoded_secret = base64.b64encode(secret.encode()).decode()
    url = f"https://example.com/profile?token={encoded_secret}&id=123"
    payload= """
    {\"body\":\"eJyVjssKwjAQRf/lrmcRlTZNfiUMJbSRlrS6EAtB+u9OgmjFjV3N484ZjnO1UpUhxB70wNjDmoYww579dAvSRVjHksMigNB1sEpGKbVZmTKvT4SCe9m6zxcvzOKne+YW6YexnVM7XC8BK2/YCMrnm7hkWlX/eaVvr+blpY+HvV7pLVbgH7EE5ic+HlWr\",\"chunk_number\":0,\"encoding\":\"zlib64\",\"request_number\":11,\"token\":\"7KSx1snskTybS6xlHnYNsAfzTHCFoGrMrYtsdcVj/Y3duTy4/64J/4IBCpc1YUAhFbgfC8BZJpZEQ8hD/a/na+/K+BmA8RLjZYPwNB2m\"}
    """
    print("\n=== Checking URL ===")
    leaks = detector.check_url(url)
    leaks_payload = detector.check_payload(payload)
    print("Leaks found in URL:", leaks)
    
    print("Leaks found in Payload:", leaks_payload)


=== Checking URL ===
URL tokens:
{'aGlfbXk', '123', 'profile', 'token', 'id'}
aGlfbXk
123
profile
token
id

URL parameters:
Tokens: set()
Parameters: set()
Leaks found in URL: [('base64', 'hi_my')]
Leaks found in Payload: []


<>:25: SyntaxWarning: invalid escape sequence '\.'
<>:381: SyntaxWarning: invalid escape sequence '\.'
<>:382: SyntaxWarning: invalid escape sequence '\.'
<>:25: SyntaxWarning: invalid escape sequence '\.'
<>:381: SyntaxWarning: invalid escape sequence '\.'
<>:382: SyntaxWarning: invalid escape sequence '\.'
/var/folders/pw/cc8d4lgn471dtgys03f89hk80000gn/T/ipykernel_36801/1621865594.py:25: SyntaxWarning: invalid escape sequence '\.'
  EXTENSION_RE = re.compile('\.[A-Za-z]{2,4}$')
/var/folders/pw/cc8d4lgn471dtgys03f89hk80000gn/T/ipykernel_36801/1621865594.py:381: SyntaxWarning: invalid escape sequence '\.'
  strings.append(re.sub('\.', '', uname))
/var/folders/pw/cc8d4lgn471dtgys03f89hk80000gn/T/ipykernel_36801/1621865594.py:382: SyntaxWarning: invalid escape sequence '\.'
  strings.append(re.sub('\.', '', uname) + '@' + domain)


In [ ]:

decodings = dict()
decodings['base16'] = lambda x: base64.b16decode(x)
decodings['base32'] = lambda x: base64.b32decode(x)
decodings['base58'] = lambda x: base58.b58decode(x)
decodings['base64'] = lambda x: base64.b64decode(x)
decodings['urlencode'] = lambda x: unquote_plus(x)
decodings['deflate'] = lambda x: _decompress_with_zlib('deflate',
                                                            x)
decodings['zlib'] = lambda x: _decompress_with_zlib('zlib', x)
decodings['gzip'] = lambda x: _decompress_with_zlib('gzip', x)
decodings['json'] = lambda x: json.loads(x)
decodings['binary'] = lambda x: ''.join(chr(int(x[i:i+8], 2)) for i in range(0, len(x), 8))
decodings['entity'] = lambda x: html.unescape(x)

decodings['rot13'] = lambda x: codecs.decode(x, 'rot_13')

for i in range(1, 26):
    if i == 13:
        continue  # handled separately with codecs
    decodings[f'rot{i}'] = lambda x, i=i: rot_n(x, i)
    
try:
    lz = LZString()
    decodings['lz64'] = lambda x: lz.decompress(base64.b64decode(x).decode('utf-8'))
except:
    pass

decodings['zlib64'] = lambda x: zlib.decompress(base64.b64decode(x)).decode('utf-8', errors='ignore')

try:
    lz = LZString()
    decodings['lz64de'] = lambda x: LZString().decompress(base64.b64decode(x).decode('utf-8'))
except:
    pass

try:
    lz = LZString()
    decodings['lz_string'] = lambda x: lz.decompressFromEncodedURIComponent(x)
except:
    pass

try: 
    import lzw
    decodings['lzw'] = lambda x: ''.join(lzw.decode(json.loads(x)))
except:
    pass

try:
    decodings['brotli_base64'] = lambda x: brotli.decompress(base64.b64decode(x)).decode('utf-8', errors='ignore')
except:
    pass

decodings['deflate_base64'] = lambda x: zlib.decompress(base64.b64decode(x), -zlib.MAX_WBITS).decode('utf-8', errors='ignore')
decodings['deflateraw_base64'] = lambda x: zlib.decompress(base64.b64decode(x), -zlib.MAX_WBITS).decode('utf-8', errors='ignore')
decodings['gzip_base64'] = lambda x: zlib.decompress(base64.b64decode(x), zlib.MAX_WBITS | 16).decode('utf-8', errors='ignore')
decodings['deflate_hex'] = lambda x: zlib.decompress(bytes.fromhex(x)).decode('utf-8', errors='ignore')
decodings['gzip_hex'] = lambda x: zlib.decompress(bytes.fromhex(x), zlib.MAX_WBITS | 16).decode('utf-8', errors='ignore')


{'base16': <function __main__.decode_or_decompress_tokens.<locals>.<lambda>(x)>,
 'base32': <function __main__.decode_or_decompress_tokens.<locals>.<lambda>(x)>,
 'base58': <function __main__.decode_or_decompress_tokens.<locals>.<lambda>(x)>,
 'base64': <function __main__.decode_or_decompress_tokens.<locals>.<lambda>(x)>,
 'urlencode': <function __main__.decode_or_decompress_tokens.<locals>.<lambda>(x)>,
 'deflate': <function __main__.decode_or_decompress_tokens.<locals>.<lambda>(x)>,
 'zlib': <function __main__.decode_or_decompress_tokens.<locals>.<lambda>(x)>,
 'gzip': <function __main__.decode_or_decompress_tokens.<locals>.<lambda>(x)>,
 'json': <function __main__.decode_or_decompress_tokens.<locals>.<lambda>(x)>,
 'binary': <function __main__.decode_or_decompress_tokens.<locals>.<lambda>(x)>,
 'entity': <function __main__.decode_or_decompress_tokens.<locals>.<lambda>(x)>,
 'rot13': <function __main__.decode_or_decompress_tokens.<locals>.<lambda>(x)>,
 'rot1': <function __main__.dec

In [12]:
import base64
import zlib

def decode_zlib64(encoded_str):
    try:
        compressed_data = base64.b64decode(encoded_str)
        decompressed_data = zlib.decompress(compressed_data)
        return decompressed_data.decode('utf-8', errors='replace')  # Replace undecodable chars
    except Exception as e:
        return f"[ERROR] {e}"

# Example usage
if __name__ == "__main__":
    zlib64_text = "eJxdjEsOhDAMQ+/idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v+TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4="  # Example input
    decoded = decode_zlib64(zlib64_text)
    print("Decoded text:\n", decoded)

Decoded text:
 [[64821,"kd",{"id":98,"m":false,"mk":[],"k":"x","cc":0,"kc":88}],[64832,"d",{"ac":[{"id":98,"a":"value","v":"hi_my_honey_tex"}]}],[64832,"k",98,"hi_my_honey_tex"]]


In [23]:


MY_SENSITIVE_ENCODINGS= {
  "tel": [
    "34393232313232323030",
    "GQ4TEMRRGIZDEMBQ",
    "3wAzuvCkRADzGP",
    "NDkyMjEyMjIwMA==",
    "4922122200",
    "33b134323234323232300000",
    "789c33b1343232343232323000000b0f01f9",
    "1f8b080000000000001333b134323234323232300000a3e481680a000000",
    "00110100001110010011001000110010001100010011001000110010001100100011000000110000",
    "4922122200",
    "4922122200",
    "M7E0MjI0MjIyMAAA",
    "M7E0MjI0MjIyMAAA",
    "H4sIAAAAAAAAEzOxNDIyNDIyMjAAAKPkgWgKAAAA",
    "iwSANDkyMjEyMjIwMAM=",
    "33b134323234323232300000",
    "1f8b080000000000001333b134323234323232300000a3e481680a000000",
    "eJwzsTQyMjQyMjIwAAALDwH5",
    "4KyE7oGM5oKM4ZiA7K2A",
    "4KyE7oGM5oKM4ZiA7K2A",
    "CwTgTGCMFgDLQ",
    "84d3a0756276940932da44e047086405",
    "7391027a4dcdd6dfa459dfe85e9cd208f7e70ca7",
    "e2ddeff7f6379160d21bf08d333a18c7b0a290c974f6451903bfc7d301b7df04",
    "d8f3d7fe7d03fdacc24d4dfca1bba976cf88450b325d25f93c10ba0f",
    "208aa443dc30af0140762b17e1f268b628fbc7a18515e68104e22eefe9a9a0e5b8e7e07a6a6b073991e66e7bc49c978d",
    "e1bf1080654ed05dd2a9c0ad2a2b61113febe705d0981fe82a6d3ff0b4c6fbfde33e8a3d5ec73341f233578d3e75a4a4394a6b3290628b9e684e0719b76f323d",
    "009700fd326c4896ad286fa000671376eacb78ddddc01c58a4381671",
    "ffbc43328e658b91b896e1a3dc5218c9549357643504cbe1ebcbe7ea0e865247",
    "69e26e31d0e4489c1717f7cc215ff3824dd359f2dc9b1cf445962a704cf5c511c878ade2887db1d39bbf5f8094b42f4e",
    "af33a8963c06306fab91e1466c8eb3da039b7149d351db1db55da38dd4f132c26597161fd0bddf002c430642357c31d92e1981f9775cf8f7acde08c1abe22e28",
    "2022247329",
    "-2687720209116599037",
    "3451519139391771119",
    "63669290229870245713155212006181530883",
    "1753343139",
    "185532921"
  ],
  "password": [
    "53616C747953656564735465613921",
    "KNQWY5DZKNSWKZDTKRSWCOJB",
    "3LJJbd4hWsNsevDznEiiU",
    "U2FsdHlTZWVkc1RlYTkh",
    "SaltySeedsTea9%21",
    "0b4ecc29a90c4e4d4d290e494db4540400",
    "789c0b4ecc29a90c4e4d4d290e494db45404002ea80576",
    "1f8b08000000000000130b4ecc29a90c4e4d4d290e494db454040084448f8b0f000000",
    "010100110110000101101100011101000111100101010011011001010110010101100100011100110101010001100101011000010011100100100001",
    "SaltySeedsTea9!",
    "FnyglFrrqfGrn9!",
    "C07MKakMTk1NKQ5JTbRUBAA=",
    "C07MKakMTk1NKQ5JTbRUBAA=",
    "H4sIAAAAAAAAEwtOzCmpDE5NTSkOSU20VAQAhESPiw8AAAA=",
    "CweAU2FsdHlTZWVkc1RlYTkhAw==",
    "0b4ecc29a90c4e4d4d290e494db4540400",
    "1f8b08000000000000130b4ecc29a90c4e4d4d290e494db454040084448f8b0f000000",
    "eJwLTswpqQxOTU0pDklNtFQEAC6oBXY=",
    "44qE44C2y6Dpu4DqmrDhjIPjoIXlmKDinIHgooA=",
    "44qE44C2y6Dpu4DqmrDhjIPjoIXlmKDinIHgooA=",
    "MoQwNgLgnsCmsBMDOAVWICcBCIA",
    "4948c713c8f51fcdf341fe5dc5f7f651",
    "486566dce40fe64622d123079598b9a0bef88551",
    "2f50045ec0f7fd7ef014aff67338010772fa21226c0b1ca4bb8a596b85a2110f",
    "2f842ab50309d0bc9db92b5921e71423cc438172313b9e3739669e06",
    "cb972f03b4d1e1032aecbc095904fd3ce72e0082498eec3fa92b5fa928ca14be586f7838b71c6c7ecde2aedaa2cfe3dc",
    "d1767a28de17b8862d8d0aebe238048082fa4d43292c941d9e45fee739beea1910d9e528174b0b4597ff9b3b9d0f59eaae00646bb25e74436836b47885bec58c",
    "12eb75e22875ddf3f4f9df97b55fcdc694a3602ce38b48c03bd709f0",
    "254b968eda5a4c4ec77b44c5f0c18486b7dcae428586446cc7e32bc4b44efd58",
    "7f2ea3fbaa3a4655eb902c84e430c2328e98655bb2eba58235067736d62137d923df8578d7dff74439ff42afd8d44ff3",
    "57ddbf73386b32c4a923a3268ec4837db4e52f84a3eca0b4e2aa6ea1dd87015fe873aa048cb84bf3f3c6790cd1f5dcfadbf67148bbe05f2664f5f6c71702dbdf",
    "-287875941",
    "2630329490964472104",
    "5021836792884838330",
    "92636538098284972449800741919914713384",
    "2341422212",
    "782763382"
  ],
  "form_text": [
    "68695F6D795F686F6E65795F74657874",
    "NBUV63LZL5UG63TFPFPXIZLYOQ======",
    "DtohHfG9wm5HUTLmENZzeX",
    "aGlfbXlfaG9uZXlfdGV4dA==",
    "hi_my_hone"
    "hi_my_honey_text",
    "cbc88ccfad8ccfc8cf4bad8c2f49ad280100",
    "789ccbc88ccfad8ccfc8cf4bad8c2f49ad280100387106bd",
    "1f8b0800000000000013cbc88ccfad8ccfc8cf4bad8c2f49ad2801002ffbe42910000000",
    "01101000011010010101111101101101011110010101111101101000011011110110111001100101011110010101111101110100011001010111100001110100",
    "hi_my_honey_text",
    "uv_zl_ubarl_grkg",
    "y8iMz62Mz8jPS62ML0mtKAEA",
    "y8iMz62Mz8jPS62ML0mtKAEA",
    "H4sIAAAAAAAAE8vIjM+tjM/Iz0utjC9JrSgBAC/75CkQAAAA",
    "iweAaGlfbXlfaG9uZXlfdGV4dAM=",
    "cbc88ccfad8ccfc8cf4bad8c2f49ad280100",
    "1f8b0800000000000013cbc88ccfad8ccfc8cf4bad8c2f49ad2801002ffbe42910000000",
    "eJzLyIzPrYzPyM9LrYwvSa0oAQA4cQa9",
    "1oTrg7rgraDpuazgvaDjrILpo4DiuYjequiAgA==",
    "1oTrg7rgraDpuazgvaDjrILpo4DiuYjequiAgA==",
    "BYSw+gtgnmwPYDsCmMAuSAeqg",
    "43e75bec2b6f2c204cb19a21baf28e34",
    "eea0bb214e955ddb5eef9c486af767e8fbc5cb39",
    "1ca570fac198727f82d0010004584b693c6e79b65c923acef84bf4169912ea33",
    "f6823a0e876191719d95ab3023ee076757ad447377d93b47abf36b90",
    "44ae82b98381ea880d1135c8ffdd52da3764fe1f1bfb693767ed25c4f6fbab8f04f675e43e5adcaa9b2bcdd07ffcda61",
    "013c9434fcc85dc02d8d50a38fc2f6495173e053b6d57b8071bd35115ff79ef8a03132d77e7e47ef0fc1e7e9873e3ae27c6b64972362751ef01b3443ad7481b3",
    "80ad41185430a92c32308e71f8d07ed84c6285a05492946e7bc3168d",
    "047ba5082a889660c46380e777a8ba2ee4e0dd4bbca69a270e67585454125b46",
    "f55210fbf2e577b3bc36bc961b960fa3963a7be7878309c847f46f0d7cc4d7b968fe0cf18ecc12b54d4fb7680c56c66a",
    "2c752ade168750df0fb478e5f8eb9fb821af98d367aee0f916402622a7bc343df8840de8a54a5d505ee96d516451ca6949ad37f0a18ad2b9c93057254e3ee0ef",
    "1753757807",
    "-2047910037623045259",
    "3847981994889785074",
    "70982739059974200961265151179235885941",
    "702872367",
    "946931389"
  ],
  "form_text2": [
    "68695F6D795F686F6E65795F6669656C64",
    "NBUV63LZL5UG63TFPFPWM2LFNRSA====",
    "yuf6pXstUhXuiBGECSh1Nzf",
    "aGlfbXlfaG9uZXlfZmllbGQ=",
    "hi_my_honey_field",
    "cbc88ccfad8ccfc8cf4bad8c4fcb4ccd490100",
    "789ccbc88ccfad8ccfc8cf4bad8c4fcb4ccd4901003f1306fc",
    "1f8b0800000000000013cbc88ccfad8ccfc8cf4bad8c4fcb4ccd4901004e4036f511000000",
    "0110100001101001010111110110110101111001010111110110100001101111011011100110010101111001010111110110011001101001011001010110110001100100",
    "hi_my_honey_field",
    "uv_zl_ubarl_svryq",
    "y8iMz62Mz8jPS62MT8tMzUkBAA==",
    "y8iMz62Mz8jPS62MT8tMzUkBAA==",
    "H4sIAAAAAAAAE8vIjM+tjM/Iz0utjE/LTM1JAQBOQDb1EQAAAA==",
    "CwiAaGlfbXlfaG9uZXlfZmllbGQD",
    "cbc88ccfad8ccfc8cf4bad8c4fcb4ccd490100",
    "1f8b0800000000000013cbc88ccfad8ccfc8cf4bad8c4fcb4ccd4901004e4036f511000000",
    "eJzLyIzPrYzPyM9LrYxPy0zNSQEAPxMG/A==",
    "1oTrg7rgraDpuazgvaDjrILpo4DmmKLkgazJpAA=",
    "1oTrg7rgraDpuazgvaDjrILpo4DmmKLkgazJpAA=",
    "BYSw+gtgnmwPYDsCmMBmIkBsAmQ",
    "f505c12230d40a5dd2538d83eedd7da7",
    "5291e16b1c4f5afef57fb6987b0c838295871e02",
    "1643c39ff8767f67ee17e7eaf7cb19ce976af8a0b9d3ecc4218111217e372f7d",
    "b78094ae62a41967e4aa69fd21bee354119d2c56ab68eaffdf197c1c",
    "3889e1e25d2983e3633edbed592734719c1c6430ec3203aceac3a487129135d20bb2ae1dcd2be66b1fbec52a1c633350",
    "8b180d4b7d90628533162567bbe3418b2f5520ee7a9de275dd3a0138cf14a7e1893bc8b525bf03772c7328d73e6972686b056b21f2bb03ca2f45c5096b2c9f83",
    "3a0c7071c21fe32fdb512689ef2e4b16b6ee7f04db7a3df039e49d4b",
    "de96f137b568833a6f68a4d358ec56bf28e3b1d1d42916a1a0f8afe690c54afe",
    "86126692cbbf6c0341274afc4c5d98afeca05a7330ef8748d491e87a2df4ae39f01128c6f349f9dccc62e1cd91bd50a9",
    "23f5e7432d6def6b1517a051f35332f199ea95fae1456242909ca845e977dee52e55110f12ee1ed16cbd139ac0332a7e3e1d0cb60d008e523598e76fa4440cc7",
    "1248393586",
    "-6414144220524007930",
    "-3909599843592213748",
    "268162979175578004603604889171042938374",
    "4113973326",
    "1058211580"
  ],
  "url_1": [
    "637572696F75732D6361742E636F6D",
    "MN2XE2LPOVZS2Y3BOQXGG33N",
    "3nPVd4R2N6xz4UUk2Zzhi",
    "Y3VyaW91cy1jYXQuY29t",
    "curious-cat.com",
    "4b2e2dcacc2f2dd64d4e2cd14bcecf0500",
    "789c4b2e2dcacc2f2dd64d4e2cd14bcecf0500302d05dd",
    "1f8b08000000000000134b2e2dcacc2f2dd64d4e2cd14bcecf0500c7c1a0530f000000",
    "011000110111010101110010011010010110111101110101011100110010110101100011011000010111010000101110011000110110111101101101",
    "curious-cat.com",
    "phevbhf-png.pbz",
    "Sy4tyswvLdZNTizRS87PBQA=",
    "Sy4tyswvLdZNTizRS87PBQA=",
    "H4sIAAAAAAAAE0suLcrMLy3WTU4s0UvOzwUAx8GgUw8AAAA=",
    "CweAY3VyaW91cy1jYXQuY29tAw==",
    "4b2e2dcacc2f2dd64d4e2cd14bcecf0500",
    "1f8b08000000000000134b2e2dcacc2f2dd64d4e2cd14bcecf0500c7c1a0530f000000",
    "eJxLLi3KzC8t1k1OLNFLzs8FADAtBd0=",
    "44aF54GO4KWg75ig7LiL5LCC4aCF7IG07IqB5rKA",
    "44aF54GO4KWg75ig7LiL5LCC4aCF7IG07IqB5rKA",
    "MYVwTglg9iDOC0wCGAXAdMKBbIA",
    "caf5110f98f91040e84a02862066fa65",
    "b7027c2cda316afe9530e01130788e901fcb7913",
    "2d216e6c59d921ffeba58dfff2b8581666172fd57bf6ee6e44ad224cb4b45a78",
    "897f1d1072385b8da1f8f3ca466d346be023f8eb335192fce69cea70",
    "a50db105ebf563ca079ecfd786f943a0e0e392b762a387ad129bc3fff3b468cc15e2ededf353c681456551eaa3bd6137",
    "9a8b60802fe42fb6aae650c391181b1e54c3a0f8efd1a180b4d698f0a6cdeceeb694c0f740cffbf49c75f0634c18283fff3c3380e5710774616b39a68a8c9bbb",
    "ef2d53be51971fb181b05ac122cd1f67407624fe9b22fd3d062d21b4",
    "0915e1bf120f2dc9d303b2a2e7a19938bfa1a78921238eb1776d2fbacd25cb5a",
    "2def97b3ff239ce0096eaa28aa320fa717f2381c7000c46cec7e0fcae6d3f475ea712b5a6b8b5915665b0d82231a5b6c",
    "cf9b6c7ac1c7f44f3eda22592edd8094e97e5d04b00f830365d024be8ad3f15c559194d8ec5bad598e567b562698ec4fe5a9a363363450b4ed2419564c1be4b8",
    "-679406168",
    "2755373884543779069",
    "-3309345380778307472",
    "279235719630208140617208984455809515773",
    "1403044295",
    "808256989"
  ],
  "url_2": [
    "68747470733A2F2F637572696F75732D6361742E636F6D",
    "NB2HI4DTHIXS6Y3VOJUW65LTFVRWC5BOMNXW2===",
    "3A8evQZovd7B4jbwRyktga3ZR6388nYC",
    "aHR0cHM6Ly9jdXJpb3VzLWNhdC5jb20=",
    "https%3A%2F%2Fcurious-cat.com",
    "cb28292928b6d2d74f2e2dcacc2f2dd64d4e2cd14bcecf0500",
    "789ccb28292928b6d2d74f2e2dcacc2f2dd64d4e2cd14bcecf0500687508a8",
    "1f8b0800000000000013cb28292928b6d2d74f2e2dcacc2f2dd64d4e2cd14bcecf0500b9e6387817000000",
    "0110100001110100011101000111000001110011001110100010111100101111011000110111010101110010011010010110111101110101011100110010110101100011011000010111010000101110011000110110111101101101",
    "https://curious-cat.com",
    "uggcf://phevbhf-png.pbz",
    "yygpKSi20tdPLi3KzC8t1k1OLNFLzs8FAA==",
    "yygpKSi20tdPLi3KzC8t1k1OLNFLzs8FAA==",
    "H4sIAAAAAAAAE8soKSkottLXTy4tyswvLdZNTizRS87PBQC55jh4FwAAAA==",
    "GxYA+MVTFpMaTJJsfhWMQGWKSyTptDo7iE8=",
    "cb28292928b6d2d74f2e2dcacc2f2dd64d4e2cd14bcecf0500",
    "1f8b0800000000000013cb28292928b6d2d74f2e2dcacc2f2dd64d4e2cd14bcecf0500b9e6387817000000",
    "eJzLKCkpKLbS108uLcrMLy3WTU4s0UvOzwUAaHUIqA==",
    "1oHnhIHshpzgroHupaDsmIXngJPohKzgvaTsoJbohITjhIDjqITjgJbsoIA=",
    "1oHnhIHshpzgroHupaDsmIXngJPohKzgvaTsoJbohITjhIDjqITjgJbsoIA=",
    "BYFxAcGcC4HpYMYFcBOBLA9kyBaBBDEAOgQwFsg",
    "c1a8de3f62604990390ff0555a524865",
    "41982bd1605b895577c73fe915e0d341c880fb60",
    "c9b19caac3fa71da9addc8270a4769744ea11f2a6a5edc27ebc1c532951cc291",
    "ed944b6e4b7a5d8d3124a20099f6126f46d398f793b9623229c108e1",
    "945e4b8f11d001863cc3ad23d239d8405484b673ba7e5f3039c7d55b32e20a72634b478f9a69f9c30beac7becf86ad8e",
    "938ca936fd2bcdff508bbea2c5665c2d4f618129c41a9fa88ad40917b9402150ee4836d26b1946d357476e0d2c7e1cad32b792faa45740474e0e7cafd94d8ffb",
    "6c901274c1162a4c9bd9a8580e06b6c2d3e4724ee62bfe89954b81cc",
    "c1d03e01aaa3442521f7d9d45fbbc02e38ce550103ac44303f511ffee6d3dc88",
    "47f9f776c15a87028a6b82e22acfd25c847c0a6d71ebf25749e0244c9957412799cb1eb83055d2c50ab94e2a86b1485e",
    "2c05c2c4dd2d3b0b967cd25fc742ac7749c058d79f913340ed51e3950f8abca92ab770ab3d3fd0af8da10b910fc5bfa94e4173891279bd7072ee53aa49d54c00",
    "1166678749",
    "2131491993515325067",
    "-5174759824637364236",
    "244824896792938886174747607789649131147",
    "2016995001",
    "1752500392"
  ],
  "form_text3": [
    "6D795F66756E6E795F686F6E6579",
    "NV4V6ZTVNZXHSX3IN5XGK6I=",
    "hFHucvJjLBAotAF5wEG",
    "bXlfZnVubnlfaG9uZXk=",
    "my_funny_honey",
    "hi_my_honey_tex",
    "cbad8c4f2bcdcbab8ccfc8cf4bad0400",
    "789ccbad8c4f2bcdcbab8ccfc8cf4bad04002cb905f8",
    "1f8b0800000000000013cbad8c4f2bcdcbab8ccfc8cf4bad0400ef8be65d0e000000",
    "0110110101111001010111110110011001110101011011100110111001111001010111110110100001101111011011100110010101111001",
    "my_funny_honey",
    "zl_shaal_ubarl",
    "y62MTyvNy6uMz8jPS60EAA==",
    "y62MTyvNy6uMz8jPS60EAA==",
    "H4sIAAAAAAAAE8utjE8rzcurjM/Iz0utBADvi+ZdDgAAAA==",
    "iwaAbXlfZnVubnlfaG9uZXkD",
    "cbad8c4f2bcdcbab8ccfc8cf4bad0400",
    "1f8b0800000000000013cbad8c4f2bcdcbab8ccfc8cf4bad0400ef8be65d0e000000",
    "eJzLrYxPK83Lq4zPyM9LrQQALLkF+A==",
    "4raE74O62aDquIfmj6DgrIPto4DqmKIA",
    "4raE74O62aDquIfmj6DgrIPto4DqmKIA",
    "LYTw+gZgrgdj4AsD2MCmIg",
    "30247d2340349e67ba677a358aeb51c2",
    "f85059d378904b17af1af3dffd1c006eb3a0d5a9",
    "dbd3135a7d724f50502c59fe740c4129ccf538b3b0a936a9aa0323e867f86cbb",
    "a34d74db44abd71b5f546ab366494f2cc808c2121f898d436e02c457",
    "99b4ff2d3c53f20a5a1815bcabeed1cfc66659fb0e9a960981ff981e4a7b808a496ec646ff1d9e7893dc7b5ed845ff11",
    "1691253552841a376371107fdf3628b9432dac01a05f3cc041875adeedddba886fd0765d9dac2e565452d82d5fb1d9602d4d971f7b74da71ae7eef0610f91270",
    "31be1bb1e2a65f888b19aafe249c9131d62508aacd81981593218561",
    "dbaa7e4a281ef017b87ab8afab5bd2b4f741e94a1106d2a6a97fd75d8550ac6e",
    "e34c0002d6a3da03d275ee17afd512629a95bef472bbe16272212d490c1b90ed037cc8821beea3283c260844f15bbda3",
    "373297e145c92a81a9de0216cf2cba7acd4626dd0537afead6e1db2c37b634e6f4c19ad59382959398f992c60207467b80a6339bcb2848a516f7eeff26f70cee",
    "-1229217392",
    "-8568163609771675660",
    "-511827998734884535",
    "330840806818517112295286193563323428852",
    "1575390191",
    "750323192"
  ]
}

MIN_LENGTH_OF_FIELD = 10 # Minimum length of field to be considered a token

def find_matches(sensitive_dict, candidates):
    matches = set()
    candidate_strs = list(map(str, candidates))  # Ensure all are strings
    concatenated = ' '.join(candidate_strs)      # Concatenate once for faster lookup

    print(type(concatenated))
    print('------')
    print('------')
    print('------')
    print("*********")
    print("*********")
    print("*********") 
    
    for key, values in sensitive_dict.items():
        for val in values:
            val_str = str(val)
            print(f"Checking value: {val_str}")
            if val_str in concatenated:
                if isinstance(key, (str, int, float)):
                    matches.add(str(key))
                else:
                    print(f"Skipping unhashable key: {key} (type: {type(key)})")
                break  # No need to check more values for this key
    return list(matches)
 
candidates = ["""
Concatenated candidates: eJxdjEsasdsadOhDAMQ /idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4 eJxdjEsasdsadOhDAMQ+/idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v+TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4 rWkqwRfnfqfnqBuQNZD+/vqELmmNvIKXJXdtdXDBJm4PZD4h4RAvOJpJj/J5i+GsVykNX0bF7NzFT04AV3D1NIjIL0O2ZOVp/OU331TYZYasjiVIl4I9srX16M2GqGHT5JKqJhKI3Iq2S1L1vjl2Zttx7z3ER5NP6NYj4 86db69b3ffe6a27a2b6a295e7607ac868a627f2b2f71ca26fee9ecb5a6e57bfa6ba1db9c79b6ad721789c5d8c4b1ab1db1a74e84300c43efe2751633cc08955ca58aaa0a8a40e5b3e02310e2ee04362056716c3f5b9bfe4df225c402b4a12ec09921b4e0d237435015c1563407630121cfc11f7df518b30b9dfc2f215cb857d7de2b5e99d93753506e565dd5ae5d5dd57761756358b0cb6320824ee6dd1139002e802f0e https://monorail-edge.shopifysvc.com/unstable/produce_batch?eJxdjEsasdsadOhDAMQ /idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4= https://monorail-edge.shopifysvc.com/unstable/produce_batch?eJxdjEsasdsadOhDAMQ+/idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v+TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4= uggcf://zbabenvy-rqtr.fubcvslfip.pbz/hafgnoyr/cebqhpr_ongpu?rWkqwRfnfqfnqBuQNZD+/vqELmmNvIKXJXdtdXDBJm4PZD4h4RAvOJpJj/J5i+GsVykNX0bF7NzFT04AV3D1NIjIL0O2ZOVp/OU331TYZYasjiVIl4I9srX16M2GqGHT5JKqJhKI3Iq2S1L1vjl2Zttx7z3ER5NP6NYj4= a6ba1db9c79b6ad721 produce_batch cebqhpr_ongpu eJxdjEsasdsadOhDAMQ /idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4= eJxdjEsasdsadOhDAMQ+/idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v+TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4= rWkqwRfnfqfnqBuQNZD+/vqELmmNvIKXJXdtdXDBJm4PZD4h4RAvOJpJj/J5i+GsVykNX0bF7NzFT04AV3D1NIjIL0O2ZOVp/OU331TYZYasjiVIl4I9srX16M2GqGHT5JKqJhKI3Iq2S1L1vjl2Zttx7z3ER5NP6NYj4= 6acb1a75e27176312c3a10c0310fbf89d458cf30225572962aa82a290396cf808c438bb810d88159c5b0fd6e6ff937c897100ad284bb026486d38348dd0d40570558d01d8c04873f047df7d462cc2e77f0bc8572e15f5f78ad7a6764dd4d41b9597756b9757755dd85d58d62c32d8c82093b9b7444e400ba00bc38 assad=eJxdjEsOhDAMQ /idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4= assad=eJxdjEsOhDAMQ+/idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v+TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4= nffnq=rWkqwRfBuQNZD+/vqELmmNvIKXJXdtdXDBJm4PZD4h4RAvOJpJj/J5i+GsVykNX0bF7NzFT04AV3D1NIjIL0O2ZOVp/OU331TYZYasjiVIl4I9srX16M2GqGHT5JKqJhKI3Iq2S1L1vjl2Zttx7z3ER5NP6NYj4= 789c5d8c4b0e84300c43efe2751633cc08955ca58aaa0a8a40e5b3e02310e2ee04362056716c3f5b9bfe4df225c402b4a12ec09921b4e0d237435015c1563407630121cfc11f7df518b30b9dfc2f215cb857d7de2b5e99d93753506e565dd5ae5d5dd57761756358b0cb6320824ee6dd1139002e802f0e eJxdjEsOhDAMQ /idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4= eJxdjEsOhDAMQ+/idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v+TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4= rWkqwRfBuQNZD+/vqELmmNvIKXJXdtdXDBJm4PZD4h4RAvOJpJj/J5i+GsVykNX0bF7NzFT04AV3D1NIjIL0O2ZOVp/OU331TYZYasjiVIl4I9srX16M2GqGHT5JKqJhKI3Iq2S1L1vjl2Zttx7z3ER5NP6NYj4= [[64821,"kd",{"id":98,"m":false,"mk":[],"k":"x","cc":0,"kc":88}],[64832,"d",{"ac":[{"id":98,"a":"value","v":"hi_my_honey_tex"}]}],[64832,"k",98,"hi_my_honey_tex"]] ['6acb1a75e27176312c3a10c0310fbf89d458cf30225572962aa82a290396cf808c438bb810d88159c5b0fd6e6ff937c897100ad284bb026486d38348dd0d40570558d01d8c04873f047df7d462cc2e77f0bc8572e15f5f78ad7a6764dd4d41b9597756b9757755dd85d58d62c32d8c82093b9b7444e400ba00bc38', 'assad=eJxdjEsOhDAMQ /idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4=', 'assad=eJxdjEsOhDAMQ+/idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v+TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4=', 'nffnq=rWkqwRfBuQNZD+/vqELmmNvIKXJXdtdXDBJm4PZD4h4RAvOJpJj/J5i+GsVykNX0bF7NzFT04AV3D1NIjIL0O2ZOVp/OU331TYZYasjiVIl4I9srX16M2GqGHT5JKqJhKI3Iq2S1L1vjl2Zttx7z3ER5NP6NYj4='] https://monorail-edge.shopifysvc.com/unstable/produce_batch?eJxdjEsasdsadOhDAMQ+/idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v+TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4= assad=eJxdjEsOhDAMQ+/idRYzzAiVXKWKqgqKQOWz4CMQ4u4ENiBWcWw/W5v+TfIlxAK0oS7AmSG04NI3Q1AVwVY0B2MBIc/BH331GLMLnfwvIVy4V9feK16Z2TdTUG5WXdWuXV3Vd2F1Y1iwy2Mggk7m3RE5AC6ALw4=
"""]

find_matches(MY_SENSITIVE_ENCODINGS, candidates)

<class 'str'>
------
------
------
*********
*********
*********
Checking value: 34393232313232323030
Checking value: GQ4TEMRRGIZDEMBQ
Checking value: 3wAzuvCkRADzGP
Checking value: NDkyMjEyMjIwMA==
Checking value: 4922122200
Checking value: 33b134323234323232300000
Checking value: 789c33b1343232343232323000000b0f01f9
Checking value: 1f8b080000000000001333b134323234323232300000a3e481680a000000
Checking value: 00110100001110010011001000110010001100010011001000110010001100100011000000110000
Checking value: 4922122200
Checking value: 4922122200
Checking value: M7E0MjI0MjIyMAAA
Checking value: M7E0MjI0MjIyMAAA
Checking value: H4sIAAAAAAAAEzOxNDIyNDIyMjAAAKPkgWgKAAAA
Checking value: iwSANDkyMjEyMjIwMAM=
Checking value: 33b134323234323232300000
Checking value: 1f8b080000000000001333b134323234323232300000a3e481680a000000
Checking value: eJwzsTQyMjQyMjIwAAALDwH5
Checking value: 4KyE7oGM5oKM4ZiA7K2A
Checking value: 4KyE7oGM5oKM4ZiA7K2A
Checking value: CwTgTGCMFgDLQ
Checking value: 84d3a07562769

['form_text3']